In [ ]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import random
import xarray as xr
from src.field_topology.topology_utils import *
import plotly.graph_objects as go

# --------------------------
# SETTINGS
# --------------------------
case = "RPS"
step = "350000"  # 115000 or 350000

if float(step) <= 115000.0:
    input_folder = f"/Volumes/data_backup/mercury/extreme/{case}_Base/10/out/"
    ncfile = os.path.join(input_folder, f"Amitis_{case}_Base_{step}.nc")
else:
    input_folder = f"/Volumes/data_backup/mercury/extreme/High_HNHV/{case}_HNHV/10/out/"
    ncfile = os.path.join(input_folder, f"Amitis_{case}_HNHV_{step}.nc")

output_folder = f"/Users/danywaller/Projects/mercury/extreme/bfield_dipole/"
os.makedirs(output_folder, exist_ok=True)

# Planet parameters
RM = 2440.0          # Mercury radius [km]
RC = 2400.0          # depth within conductive layer [km]

if case in ["RPS", "RPN"]:
    plot_depth = RM
elif case in ["CPS", "CPN"]:
    plot_depth = RM
else:
    raise ValueError("Invalid case ID, pick one of RPS, RPN, CPS, CPN")

# Seed settings
n_lat = 45
n_lon = n_lat*2
max_steps = 5000
h_step = 75.0
surface_tol = -75.0

max_lines = 5000  # downsample trajectory points for plotting


In [ ]:
# --------------------------
# CREATE SEEDS ON SPHERE
# --------------------------
lats_surface = np.linspace(-90, 90, n_lat)
lons_surface = np.linspace(-180, 180, n_lon)
seeds = []
for lat in lats_surface:
    for lon in lons_surface:
        phi = np.radians(lat)
        theta = np.radians(lon)
        x_s = plot_depth*np.cos(phi)*np.cos(theta)
        y_s = plot_depth*np.cos(phi)*np.sin(theta)
        z_s = plot_depth*np.sin(phi)
        seeds.append(np.array([x_s, y_s, z_s]))
seeds = np.array(seeds)

print(f"Generated {len(seeds)} seeds")


In [ ]:
# --------------------------
# LOAD VECTOR FIELD FROM NETCDF
# --------------------------
def load_field(ncfile):
    ds = xr.open_dataset(ncfile)
    x = ds["Nx"].values
    y = ds["Ny"].values
    z = ds["Nz"].values

    # Extract fields (drop time dimension)
    Bx = ds["Bdx"].isel(time=0).values
    By = ds["Bdy"].isel(time=0).values
    Bz = ds["Bdz"].isel(time=0).values
    ds.close()

    #  Transpose: Nz, Ny, Nx --> Nx, Ny, Nz
    Bx_plane = np.transpose(Bx, (2, 1, 0))
    By_plane = np.transpose(By, (2, 1, 0))
    Bz_plane = np.transpose(Bz, (2, 1, 0))

    return x, y, z, Bx_plane, By_plane, Bz_plane

x, y, z, Bx, By, Bz = load_field(ncfile)

start = datetime.now()
print(f"Loaded {ncfile} at {str(start)}")


In [ ]:
# --------------------------
# TRACE FIELD LINES
# --------------------------
lines_by_topo = {"closed": [], "open": []}

for seed in seeds:
    traj_fwd, exit_fwd_y = trace_field_line_rk(seed, Bx, By, Bz, x, y, z, plot_depth, max_steps=max_steps, h=h_step, surface_tol=surface_tol)
    traj_bwd, exit_bwd_y = trace_field_line_rk(seed, Bx, By, Bz, x, y, z, plot_depth, max_steps=max_steps, h=-h_step, surface_tol=surface_tol)
    topo = classify(traj_fwd, traj_bwd, plot_depth, exit_fwd_y, exit_bwd_y)
    if topo in ["closed", "open"]:
        lines_by_topo[topo].append(traj_fwd)
        lines_by_topo[topo].append(traj_bwd)

classtime = datetime.now()
print(f"Classified all lines at {str(classtime)}")


In [ ]:
# function to lightly smooth field lines
from scipy.signal import savgol_filter

def smooth_traj(traj, k=5, order=2):
    if traj.shape[0] < k:
        return traj

    return np.column_stack([
        savgol_filter(traj[:, i], k, order, mode="interp")
        for i in range(3)
    ])
    
# --------------------------
# PLOT 3D FIELD LINES
# --------------------------
colors = {"closed": "blue", "open": "red"}
fig = go.Figure()

# add planet sphere
theta = np.linspace(0, np.pi, 100)        # colatitude
phi   = np.linspace(0, 2*np.pi, 200)      # longitude
theta, phi = np.meshgrid(theta, phi)

xs = plot_depth * np.sin(theta) * np.cos(phi)
ys = plot_depth * np.sin(theta) * np.sin(phi)
zs = plot_depth * np.cos(theta)

eps = 0
mask_pos = xs >= -eps
mask_neg = xs <=  eps

# light grey hemisphere (X > 0)
fig.add_trace(go.Surface(
    x=np.where(mask_pos, xs, np.nan),
    y=np.where(mask_pos, ys, np.nan),
    z=np.where(mask_pos, zs, np.nan),
    surfacecolor=np.ones_like(xs),
    colorscale=[[0, 'lightgrey'], [1, 'lightgrey']],
    cmin=0,
    cmax=1,
    showscale=False,
    lighting=dict(ambient=1, diffuse=0, specular=0),
    hoverinfo='skip'
))

# black hemisphere (X <= 0)
fig.add_trace(go.Surface(
    x=np.where(mask_neg, xs, np.nan),
    y=np.where(mask_neg, ys, np.nan),
    z=np.where(mask_neg, zs, np.nan),
    surfacecolor=np.zeros_like(xs),
    colorscale=[[0, 'black'], [1, 'black']],
    cmin=0,
    cmax=1,
    showscale=False,
    lighting=dict(ambient=1, diffuse=0, specular=0),
    hoverinfo='skip'
))

# add field lines
for topo, lines in lines_by_topo.items():
    first = True  # flag to show legend only once per topo

    # Downsample lines if there are too many
    if len(lines) > max_lines:
        lines_to_plot = random.sample(lines, max_lines)
    else:
        lines_to_plot = lines

    for traj in lines_to_plot:
        # traj_s = smooth_traj(traj)
        traj_s = traj

        fig.add_trace(go.Scatter3d(
            x=traj_s[:, 0],
            y=traj_s[:, 1],
            z=traj_s[:, 2],
            mode='lines',
            line=dict(color=colors[topo], width=2),
            name=topo,
            legendgroup=topo,
            showlegend=first
        ))
        first = False  # only first trace per topo shows in legend

fig.update_layout(
    template="plotly",
    width=1000,
    height=800,
    scene=dict(
            xaxis=dict(title='X [km]', range=[-10 * RM, 10 * RM]),
            yaxis=dict(title='Y [km]', range=[-10 * RM, 10 * RM]),
            zaxis=dict(title='Z [km]', range=[-10 * RM, 10 * RM]),
            aspectmode='cube',
        ),
    legend=dict(
        groupclick="togglegroup"
    ),
    title=f"{case} Dipole Field Line Topology at t = {float(step)*0.002} s"
)

out_html = f"{case}_{step}_dipole_vector_topology.html"
fig.write_html(os.path.join(output_folder, out_html), include_plotlyjs="cdn")
fig.write_image(os.path.join(output_folder, out_html.replace(".html", ".png")), scale=2)
plottime = datetime.now()
print(f"Saved figure at {str(plottime)}")
